In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import time
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import random
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression

history = pd.read_csv('data/history.tsv', sep='\t')
users = pd.read_csv('data/users.tsv', sep='\t')
validate = pd.read_csv('data/validate.tsv', sep='\t')
validate_answers = pd.read_csv('data/validate_answers.tsv', sep='\t')

In [31]:
users.user_id.tail()

27764    27764
27765    27765
27766    27766
27767    27767
27768    27768
Name: user_id, dtype: int64

In [3]:
users.head()

,user_id,sex,age,city_id
0,0,2,19,0
1,1,1,0,1
2,2,2,24,2
3,3,1,20,3
4,4,2,29,4


In [2]:
df = validate.copy()
df['hours'] = df.hour_end - df.hour_start
df['at_least_one'] = validate_answers.at_least_one.copy()
df['at_least_two'] = validate_answers.at_least_two.copy()
df['at_least_three'] = validate_answers.at_least_three.copy()

In [53]:
check_df = df[['cpm', 'hours', 'average_fit', 'average_active', 'at_least_one']].copy()

In [54]:
sort_res = check_df.sort_values('at_least_one', ascending=False)

In [56]:
sort_res.corr()

,cpm,hours,average_fit,average_active,at_least_one
cpm,1.000000,-0.011378,0.001494,0.003511,0.199448
hours,-0.011378,1.000000,-0.021334,0.060703,0.480316
average_fit,0.001494,-0.021334,1.000000,0.089754,0.548116
average_active,0.003511,0.060703,0.089754,1.000000,0.358690
at_least_one,0.199448,0.480316,0.548116,0.358690,1.000000


In [58]:
sort_res.head(20)

,cpm,hours,average_fit,average_active,at_least_one
831,130.0,422,0.931105,0.132664,0.9307
939,307.0,54,0.927132,0.131910,0.9016
629,100.0,271,0.939045,0.130310,0.8819
226,90.0,220,0.923184,0.131209,0.8500
376,123.0,215,0.857986,0.085701,0.7743
583,145.0,242,0.717645,0.079710,0.6883
473,160.0,146,0.635970,0.129796,0.6576
901,80.0,243,0.663628,0.132373,0.6500
771,90.0,225,0.689561,0.131688,0.6474
556,330.0,447,0.610457,0.081979,0.6392


In [63]:
one = 0.132664
(sort_res.average_fit * abs(one - average_active) - sort_res.at_least_one).mean()

-0.07309189475328766

In [87]:
(sort_res.average_fit - sort_res.at_least_one).mean()

0.29333605238429206

In [110]:
h = (sort_res.hours - sort_res.hours.mean()) / sort_res.hours.std()
g = (sort_res.cpm - sort_res.cpm.mean()) / sort_res.cpm.std()
a = sort_res.average_active.to_numpy()
a = (a - a.mean()) / a.std()
t = sort_res.average_fit.to_numpy()#.reshape(1, -1)
s = sort_res.at_least_one.to_numpy()#.reshape(1, -1)
t.shape, s.shape

((1008,), (1008,))

In [85]:
X = list(map(list, zip(t, a)))
y = s

In [91]:
a

array([0.13266426, 0.13191048, 0.13030988, ..., 0.01218533, 0.0256968 ,
       0.02780498])

In [144]:
for i in map(list, zip(a, h)):
    print(i)

[-0.09905904758218081, -0.048882231598838165]
[2.5040624368397975, -0.7530124116237207]
[0.0013383342702241563, -0.6422503608332897]
[-0.1550303886018028, -0.15173270733280977]
[-0.24751442612771388, 1.0824730014748494]
[-0.02214157589850612, -0.5314883100428587]
[-0.295019876561008, 1.2011466273217395]
[-0.00554960188911626, -0.47610728464764324]
[-0.7969951428241698, -0.673896661059127]
[-0.12090456374700935, -0.784658711849558]
[-0.10329309288569499, -0.5393998850993181]
[-0.12578347241518717, 1.0508267012490118]
[-0.18704216910724147, -0.7292776864543425]
[-0.03451096666560062, 0.9242415003456623]
[0.07941834261313623, 0.29131549582891403]
[-0.29074623389616505, -0.7134545363414239]
[-0.0629833924327843, -0.5473114601557775]
[-0.9444003750067678, -0.7688355617366394]
[-0.44307731453365523, -0.0014127812600820435]
[0.060354229838230314, -0.7530124116237207]
[-0.06267527029605931, -0.784658711849558]
[-0.48745856651584113, 0.6315132232566663]
[-0.4840666500385033, -0.7767471367930987

In [158]:
h = (check_df.hours - check_df.hours.mean()) / check_df.hours.std()
g = (check_df.cpm - check_df.cpm.mean()) / check_df.cpm.std()
a = check_df.average_active.to_numpy()
a = (a - a.mean()) / a.std()
t = check_df.average_fit.to_numpy()#.reshape(1, -1)
s = check_df.at_least_one.to_numpy()#.reshape(1, -1)
t.shape, s.shape
X = list(map(list, zip(a)))
y = s - t
reg = LinearRegression().fit(X, y)
((t + np.array([reg.predict([input_]) for input_ in map(list, zip(a))]) - s) ** 2).mean()

0.0433714940962735

In [152]:
t + np.array([reg.predict([input_]) for input_ in map(list, zip(a))]) - s

array([[-0.26178877, -0.25581158,  0.37712672, ..., -0.05977072,
         0.07229105,  0.24612476],
       [-0.18286669, -0.1768895 ,  0.4560488 , ...,  0.01915136,
         0.15121313,  0.32504684],
       [-0.2587449 , -0.25276771,  0.38017059, ..., -0.05672685,
         0.07533492,  0.24916863],
       ...,
       [-0.19456409, -0.1885869 ,  0.4443514 , ...,  0.00745396,
         0.13951573,  0.31334944],
       [-0.26661978, -0.26064259,  0.37229571, ..., -0.06460173,
         0.06746004,  0.24129375],
       [-0.26208256, -0.25610537,  0.37683293, ..., -0.06006451,
         0.07199726,  0.24583097]])

In [150]:
reg.coef_

array([0.03031825])

In [74]:
df[['hours', 'cpm', 'at_least_one', 'at_least_two', 'at_least_three']].corr()

,hours,cpm,at_least_one,at_least_two,at_least_three
hours,1.000000,-0.011378,0.480316,0.534585,0.547955
cpm,-0.011378,1.000000,0.199448,0.150554,0.132197
at_least_one,0.480316,0.199448,1.000000,0.962787,0.918242
at_least_two,0.534585,0.150554,0.962787,1.000000,0.986001
at_least_three,0.547955,0.132197,0.918242,0.986001,1.000000


In [45]:
average_active = np.array([np.mean(df.iloc[i, :].users_overall_activity) for i in range(1008)])
average_active.max(), average_active.argmax()

(0.1332422341696535, 116)

In [46]:
#np.mean(df.iloc[116, :].publ_fit_users)
df.iloc[116, :]

cpm                                                                     399
hour_start                                                             1049
hour_end                                                               1290
publishers                                                           3,8,10
audience_size                                                           900
user_ids                  20,38,103,104,114,133,222,242,264,305,306,346,...
hours                                                                   241
at_least_one                                                         0.0944
at_least_two                                                         0.0756
at_least_three                                                       0.0644
publ_fit_users            [0.0, 0.0, 0.0, 0.0, 0.05517241379310345, 0.0,...
users_overall_activity    [0.11895161290322581, 0.12029569892473119, 0.2...
estim_activity            [28.66733870967742, 28.991263440860216, 71.911...
Name: 116, d

In [46]:
df['average_fit'] = average_fit
df['average_active'] = average_active

In [43]:
average_fit = np.array([np.mean(df.iloc[i, :].publ_fit_users) for i in range(1008)])
#get_smoothed_mean_log_accuracy_ratio(df.at_least_one, average_fit)

In [3]:
user_saw_ads = history.groupby('user_id').size()
user_publishers = np.zeros((27768 + 1, 22))
for i, row in history.iterrows():
    user_publishers[int(row.user_id)][int(row.publisher)] += 1.0 / user_saw_ads[row.user_id]

def make_publ_fit_users(row):
    users_in_row = pd.Series(row.user_ids.split(',')).apply(int)
    publishers_in_row = pd.Series(row.publishers.split(',')).apply(int)
    
    feater = []
    for current_user in users_in_row:
        user_sum = 0
        for current_publisher in publishers_in_row:
            user_sum += user_publishers[current_user][current_publisher]
        feater.append(user_sum)
    return np.array(feater)

df['publ_fit_users'] = df.apply(make_publ_fit_users, axis=1)

In [4]:
#users_hours = np.zeros((27768 + 1, 1491))
#for i, row in history.iterrows():
#    users_hours[int(row.user_id)][int(row.hour)] = 1
#users_overall_active = pd.Series(map(sum, users_hours)) / 1488

users_overall_active_alt = history.groupby('user_id').size().reindex(range(27768 + 1), fill_value=0) / 1488 #will
                                                                                                    #scale anyway
def make_users_overall_activity(users_in_row):
    tmp = pd.Series(users_in_row.split(',')).apply(int)
    feater = []
    for current_user in tmp:
        feater.append(users_overall_active_alt[current_user])
    return np.array(feater)

df['users_overall_activity'] = df.user_ids.apply(make_users_overall_activity)
df['estim_activity'] = df.hours * df.users_overall_activity

In [ ]:
sex_publishers = np.zeros((22, 3))
for i, row in history.iterrows():
    sex_publishers[int(row.publisher)][users.iloc[int(row.user_id), 1]] += 1 # not unique
sex_ratio = [sex_publishers[i][1] / (sex_publishers[i][1] + sex_publishers[i][2]) for i in range(22)] 

In [30]:
def make_sex_ratio(row):
    users_in_row = pd.Series(row.user_ids.split(',')).apply(int)
    publishers_in_row = pd.Series(row.publishers.split(',')).apply(int)
    
    feater = []
    for current_user in users_in_row:
        max_match = 0
        for current_publisher in publishers_in_row:
            if users.iloc[int(current_user), 1] == 1:
                max_match = max(max_match, sex_ratio[current_publisher])
            elif users.iloc[int(current_user), 1] == 2:
                max_match = max(max_match, 1 - sex_ratio[current_publisher])
        feater.append(max_match)
    return np.array(feater)
df['sex_ratio'] = df.apply(make_sex_ratio, axis=1)

In [7]:
age_publisher = [[] for _ in range(22)]
for i, row in history.iterrows():
    age = users.iloc[int(row.user_id), 2]
    if age <= 10 or age > 80:
        continue
    age_publisher[int(row.publisher)].append(age) # todo NOT unique users now, try unique
average_age = [np.array(age_publisher[i]).mean() for i in range(22)]

/home/andreybocharnikov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice.
  import sys
/home/andreybocharnikov/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [530]:
average_age

[nan,
 28.483439431537267,
 26.034593899449423,
 32.26166913426323,
 31.327407407407406,
 29.178716744913928,
 25.059045226130653,
 33.008711179346825,
 33.89227166276346,
 35.900501226405034,
 36.56395200408476,
 31.22362024592508,
 34.57267441860465,
 28.173410404624278,
 31.743515850144092,
 30.55971479500891,
 30.53464566929134,
 29.65953488372093,
 31.98830409356725,
 20.442307692307693,
 30.32941176470588,
 20.0]

In [ ]:
age_publisher = np.zeros((22, 27768 + 1))
for i, row in history.iterrows():
    age = users.iloc[int(row.user_id), 2]
    age_publisher[int(row.publisher)][int(row.user_id)] = 1.0 # Unique users now

average_age = np.zeros(22)
for i in range(22):
    ages = []
    for j in range(27768 + 1):
        if age_publisher[i][j]:
            ages.append(users.iloc[j, 2])
    average_age[i] = np.mean(ages)

In [419]:
average_age

array([        nan, 28.14754311, 27.55918505, 28.66854865, 30.29093369,
       27.25      , 25.63551402, 30.72772809, 31.72463768, 34.49608355,
       35.98936978, 30.4962406 , 32.28827038, 24.5       , 32.09448819,
       25.96969697, 29.176781  , 30.19047619, 26.95      , 24.51162791,
       31.47826087, 23.5       ])

In [8]:
def make_average_age(row):
    users_in_row = pd.Series(row.user_ids.split(',')).apply(int)
    publishers_in_row = pd.Series(row.publishers.split(',')).apply(int)
    
    feater = []
    for current_user in users_in_row:
        min_dif = 100
        for current_publisher in publishers_in_row:
            min_dif = min(min_dif, abs(users.iloc[int(current_user), 2] - average_age[current_publisher]))
        feater.append(min_dif)
    return np.array(feater)
df['average_age'] = df.apply(make_average_age, axis=1)

In [20]:
df.head(7)

,cpm,hour_start,hour_end,publishers,audience_size,user_ids,hours,at_least_one,at_least_two,at_least_three,publ_fit_users,users_overall_activity,estim_activity
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",95,0.0430,0.0152,0.0073,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0006720430107526882, 0.013440860215053764, ...","[0.06384408602150538, 1.2768817204301075, 0.0,..."
1,312.0,1295,1301,"3,18",1380,"29,81,98,102,165,167,195,205,218,231,242,263,3...",6,0.0130,0.0000,0.0000,"[0.1702127659574468, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.031586021505376344, 0.1364247311827957, 0.0...","[0.18951612903225806, 0.8185483870967741, 0.36..."
2,70.0,1229,1249,"1,2,3,9,15,21",888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5...",20,0.0878,0.0135,0.0000,"[0.0, 1.0000000000000002, 0.9999999999999996, ...","[0.0006720430107526882, 0.03696236559139785, 0...","[0.013440860215053765, 0.739247311827957, 0.32..."
3,240.0,1295,1377,"1,14",440,"44,122,187,209,242,255,312,345,382,465,513,524...",82,0.2295,0.1295,0.0727,"[1.0000000000000002, 0.9999999999999987, 1.000...","[0.013440860215053764, 0.07056451612903226, 0....","[1.1021505376344085, 5.786290322580646, 0.9919..."
4,262.0,752,990,"1,3,7,8",1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2...",238,0.3963,0.2785,0.2270,"[1.0, 0.9074074074074069, 1.0000000000000002, ...","[0.0020161290322580645, 0.036290322580645164, ...","[0.4798387096774193, 8.63709677419355, 5.75806..."
5,141.0,1288,1322,"3,8,11,15",856,"111,217,235,247,292,340,398,540,541,546,581,61...",34,0.0187,0.0035,0.0012,"[0.0, 0.0, 0.6842105263157894, 0.0, 0.0, 0.0, ...","[0.038306451612903226, 0.002688172043010753, 0...","[1.3024193548387097, 0.0913978494623656, 0.434..."
6,250.0,1198,1451,"1,3,7,10,13,18",336,"24,43,96,124,150,256,299,313,430,450,552,557,5...",253,0.3839,0.2917,0.2202,"[0.981481481481481, 0.9999999999999994, 0.0, 0...","[0.036290322580645164, 0.030913978494623656, 0...","[9.181451612903226, 7.821236559139785, 0.0, 6...."


In [10]:
norm = []
sizes = []
for i, curr_row in validate.iterrows():
    cpms = []
    for j, row in validate.iterrows():
        if j == i:
            cpms.append(row.cpm)
            continue
        if row.hour_start >= curr_row.hour_end or row.hour_end <= curr_row.hour_start:
            continue
        crossing = min(curr_row.hour_end - row.hour_start, row.hour_end - curr_row.hour_start)
        # print(curr_row, row, crossing)
        if (3 * crossing >= curr_row.hour_end - curr_row.hour_start):
            cpms.append(row.cpm)
    tmp = np.array(cpms)
    # print(tmp)
    sizes.append(tmp.size)
    norm.append((curr_row.cpm - tmp.mean()) / tmp.std())
df['cpm_norm'] = norm

In [282]:
# ad has max cpm times
max_cpm = np.zeros(validate.shape[0])
start = validate.hour_start.min()
end = validate.hour_end.max()
for i in range(start, end + 1):
    index = validate[(validate.hour_start <= i) & (validate.hour_end >= i)].cpm.idxmax()
    max_cpm[index] += 1
df['max_cpm'] = max_cpm
df.max_cpm = (df.max_cpm - df.max_cpm.mean()) / df.max_cpm.std()

In [83]:
start = time.time()
user_in_ad = [[] for i in range(27769)]
#user_cpm = np.zeros(27769)
for i, row in validate.iterrows():
    users_in_row = list(map(int, (row.user_ids.split(','))))
    publishers_in_row = list(map(int, (row.publishers.split(','))))
    for user in users_in_row:
        user_in_ad[user].append([row[0], row[1], row[2], publishers_in_row]) # 0 - cpm, 1 - start, 2 - end
        
        #user_in_ad[user].append(row)
#for i in range(27769):
#    user_cpm[i] = np.mean(user_in_ad[user])

#for i in range(27769):
    #user_in_ad[i] = sorted(user_in_ad[i], key=lambda x: x[1], reverse=True)    
    #for user_ad in user_in_ad[i]:
        
time.time() - start

34.93978023529053

In [95]:
np.array(list(map(lambda x: len(x), user_in_ad))).argmax()

11105

In [106]:
validate.iloc[831, :]

cpm                                                            130
hour_start                                                     957
hour_end                                                      1379
publishers                                              1,2,7,9,16
audience_size                                                  952
user_ids         4,78,104,114,116,165,222,233,242,336,386,402,4...
Name: 831, dtype: object

In [327]:
df.iloc[2, :]

cpm                                                               -0.823809
hour_start                                                             1229
hour_end                                                               1249
publishers                                                    1,2,3,9,15,21
audience_size                                                           888
user_ids                  12,23,25,29,45,85,92,124,156,190,272,334,456,5...
hours                                                              -0.64225
at_least_one                                                         0.0878
at_least_two                                                         0.0135
at_least_three                                                            0
publ_fit_users            [0.0, 1.0000000000000002, 0.9999999999999996, ...
users_overall_activity    [0.0006720430107526882, 0.03696236559139785, 0...
estim_activity            [0.013440860215053765, 0.739247311827957, 0.32...
average_age 

In [89]:
row = validate.iloc[651, :]
users_in_row = list(map(int, (row.user_ids.split(','))))
publishers_in_row = list(map(int, (row.publishers.split(','))))
start = row.hour_start
end = row.hour_end
cpm = row.cpm
#user = 4
statistic = []
user = 15661
print("Check for user ", user)
print("cpm - ", cpm, "start - ", start, "end - ", end)
sum_hours = 0
for publ in publishers_in_row:
    print("publisher is ", publ)
    scopes = [(start, end)]
    for user_ad in user_in_ad[user]:
        if publ not in user_ad[3]:
            continue
        new_scope = False
        loc = None
        print("old scopes", scopes)
        print("user_ad ", user_ad[0], user_ad[1], user_ad[2])
        #print(user_ad)
        for it, pare in enumerate(scopes):
            if user_ad[0] <= cpm: # case of == 
                continue
            if user_ad[1] >= pare[1] or user_ad[2] <= pare[0]: # 0 -cpm, 1 - start, 2 - end
                continue
            if user_ad[1] <= pare[0] and user_ad[2] >= pare[1]:
                scopes[it] = (pare[1], pare[1])
                continue
            if user_ad[1] > pare[0] and user_ad[2] < pare[1]:
                new_scope = True
                loc = it
                scopes.append((pare[0], user_ad[1]))
                scopes.append((user_ad[2] + 1, pare[1]))
                break
            if user_ad[1] > pare[0]:
                scopes[it] = (pare[0], user_ad[1])
            elif user_ad[2] < pare[1]: # same as used_ad[1] < pare[0]
                scopes[it] = (user_ad[2], pare[1])
            else:
                print(scopes)
                print(user_ad)
                print((user_ad[1] <= pare[0] and user_ad[2] >= pare[1]))
                assert(False)

        if new_scope:
            scopes[-1], scopes[loc] = scopes[loc], scopes[-1]
            scopes.pop()
        print("new scopes ", scopes)
        print()
    sum_hour_pare = 0
    for pare in scopes:
        sum_hour_pare += pare[1] - pare[0]
    sum_hours += sum_hour_pare * user_publishers[user][publ]

#statistic.append(sum_hours)
    #print('user - ', user, 'hours - ', sum_hours)
    #print("new scopes ", scopes)
#np.mean(statistic)

Check for user  15661
cpm -  30.0 start -  1060 end -  1254
publisher is  1
old scopes [(1060, 1254)]
user_ad  280.0 1286 1292
new scopes  [(1060, 1254)]

old scopes [(1060, 1254)]
user_ad  124.0 1305 1403
new scopes  [(1060, 1254)]

old scopes [(1060, 1254)]
user_ad  148.0 1215 1347
new scopes  [(1060, 1215)]

old scopes [(1060, 1215)]
user_ad  150.0 1252 1438
new scopes  [(1060, 1215)]

old scopes [(1060, 1215)]
user_ad  134.0 1276 1379
new scopes  [(1060, 1215)]

old scopes [(1060, 1215)]
user_ad  60.0 826 850
new scopes  [(1060, 1215)]

old scopes [(1060, 1215)]
user_ad  130.0 948 1077
new scopes  [(1077, 1215)]

old scopes [(1077, 1215)]
user_ad  140.0 1178 1315
new scopes  [(1077, 1178)]

old scopes [(1077, 1178)]
user_ad  90.0 1055 1078
new scopes  [(1078, 1178)]

old scopes [(1078, 1178)]
user_ad  149.0 1160 1259
new scopes  [(1078, 1160)]

old scopes [(1078, 1160)]
user_ad  280.0 1038 1197
new scopes  [(1160, 1160)]

old scopes [(1160, 1160)]
user_ad  94.0 1138 1471
new scopes

In [84]:
start_time = time.time()
ad_user_hours = {}
for i, row in validate.iterrows():
    users_in_row = list(map(int, (row.user_ids.split(','))))
    publishers_in_row = list(map(int, (row.publishers.split(','))))
    n_publish = len(publishers_in_row)
    start = row.hour_start
    end = row.hour_end
    cpm = row.cpm
    for user in users_in_row:
        sum_hours = 0
        for publ in publishers_in_row:
            #if user_publishers[user][publ] < 0.20: # idea - count only interesting publishes; it's really fast
            #    continue
            scopes = [(start, end)]
            for user_ad in user_in_ad[user]:
                if publ not in user_ad[3]:
                    continue
                new_scope = False
                loc = None
                for it, pare in enumerate(scopes):
                    if user_ad[0] <= cpm: # case of == 
                        continue
                    if user_ad[1] >= pare[1] or user_ad[2] <= pare[0]: # 0 -cpm, 1 - start, 2 - end
                        continue
                    if user_ad[1] <= pare[0] and user_ad[2] >= pare[1]:
                        scopes[it] = (pare[1], pare[1])
                        continue
                    if user_ad[1] > pare[0] and user_ad[2] < pare[1]:
                        new_scope = True
                        loc = it
                        scopes.append((pare[0], user_ad[1]))
                        scopes.append((user_ad[2] + 1, pare[1]))
                        break
                    if user_ad[1] > pare[0]:
                        scopes[it] = (pare[0], user_ad[1])
                    elif user_ad[2] < pare[1]: # same as used_ad[1] < pare[0]
                        scopes[it] = (user_ad[2], pare[1])
                    else:
                        print(scopes, user_ad, (user_ad[1] <= pare[0] and user_ad[2] >= pare[1]))
                        assert(False)
                if new_scope:
                    scopes[-1], scopes[loc] = scopes[loc], scopes[-1]
                    scopes.pop()
            sum_hours_pare = 0
            for pare in scopes:
                sum_hours_pare += pare[1] - pare[0]
            sum_hours += sum_hours_pare * user_publishers[user][publ]
        ad_user_hours[(i, user)] = sum_hours
time.time() - start_time

79.75986194610596

In [85]:
len(ad_user_hours)

1098808

In [86]:
ad_user_hours[651, 15661] ###

0.0

In [9]:
start_time = time.time()
publisher_in_ad = [[] for i in range(27769)]
for i, row in validate.iterrows():
    users_in_row = list(map(int, (row.user_ids.split(','))))
    publishers_in_row = list(map(int, (row.publishers.split(','))))
    for publ in publishers_in_row:
        publisher_in_ad[publ].append([row[0], row[1], row[2], users_in_row]) # 0 - cpm, 1 - start, 2 - end
time.time() - start_time

0.40518617630004883

In [72]:
validate.hour_start.min()

747

In [82]:
validate_answers.iloc[651, :]

at_least_one      0.2711
at_least_two      0.1150
at_least_three    0.0489
Name: 651, dtype: float64

In [81]:
validate[(pd.Series(validate.user_ids).apply(lambda x: '15661' in x))]

,cpm,hour_start,hour_end,publishers,audience_size,user_ids
19,280.0,1286,1292,"1,7",392,"85,111,118,231,271,274,396,453,468,489,546,573..."
41,376.0,951,980,"3,7",440,"5,20,137,381,382,387,444,544,550,551,603,656,6..."
50,80.0,962,1149,"3,5",1320,"8,12,118,142,148,165,217,220,231,271,274,290,2..."
63,124.0,1305,1403,"1,2,4,7,13,16",2460,"2,10,12,34,41,42,47,48,60,74,75,79,80,87,88,95..."
80,120.0,1094,1099,"2,3,17,18,21",984,"8,111,118,145,148,217,220,295,322,375,396,402,..."
102,118.0,820,901,"2,5,11",1344,"4,5,12,15,17,42,48,65,99,106,119,125,137,140,1..."
128,148.0,1215,1347,"1,7",2360,"2,4,5,15,17,19,21,25,42,48,65,74,88,96,99,106,..."
160,346.0,1083,1166,"2,3,6,11,21",972,"4,8,10,12,42,126,128,129,174,176,200,206,234,2..."
163,74.0,1085,1485,"2,3,7",1416,"17,21,25,48,74,88,96,99,110,119,125,137,140,14..."
181,150.0,1252,1438,"1,2",2400,"4,5,12,15,17,19,21,25,42,47,48,65,74,88,96,99,..."


In [94]:
validate_answers.iloc[695, :]

at_least_one      0.4272
at_least_two      0.3349
at_least_three    0.2836
Name: 695, dtype: float64

In [97]:
history[(history.cpm == 30) & (history.hour == 1114) & (history.publisher == 3)]

,hour,cpm,publisher,user_id
732649,1114,30.0,3,27594
733159,1114,30.0,3,26159
733275,1114,30.0,3,3298
735089,1114,30.0,3,19946
735935,1114,30.0,3,9320
737052,1114,30.0,3,27594
738470,1114,30.0,3,9320
1072800,1114,30.0,3,19946
1072945,1114,30.0,3,6523


In [69]:
history[history.user_id == 15661].sort_values('hour')

,hour,cpm,publisher,user_id
0,10,30.00,1,15661
3233,10,42.50,1,15661
1044356,35,30.00,1,15661
38808,50,40.75,1,15661
73409,105,30.00,1,15661
1136176,136,210.00,1,15661
126032,206,30.00,1,15661
132182,207,30.00,1,15661
138125,224,72.50,1,15661
140189,224,30.00,1,15661


In [32]:
validate[validate.cpm == 142]

,cpm,hour_start,hour_end,publishers,audience_size,user_ids
52,142.0,996,1414,"2,4,7",1500,"0,2,10,31,35,36,70,90,96,143,207,219,232,256,2..."
498,142.0,1039,1045,"2,7,21",1620,"3,8,11,26,27,59,83,90,124,128,189,199,224,230,..."


In [43]:
validate[validate.index == 831]

,cpm,hour_start,hour_end,publishers,audience_size,user_ids
831,130.0,957,1379,"1,2,7,9,16",952,"4,78,104,114,116,165,222,233,242,336,386,402,4..."


In [27]:
start_time = time.time()
row = validate.iloc[831, :]
users_in_row = list(map(int, (row.user_ids.split(','))))
publishers_in_row = list(map(int, (row.publishers.split(','))))
start = row.hour_start
end = row.hour_end
cpm = row.cpm
print("cpm - ", cpm, "start -", start, "end - ", end)
times = []
for publ in publishers_in_row:
    sum_hours = 0
    scopes = [(start, end)]
    for publ_ad in publisher_in_ad[publ]:
        
        new_scope = False
        loc = None
        if publ == 2:
            print("old scopes ", scopes)
            print("publ_ad ", publ_ad[0], publ_ad[1], publ_ad[2])
        for it, pare in enumerate(scopes):
            if publ_ad[0] <= cpm: # case of == 
                continue
            if publ_ad[1] >= pare[1] or publ_ad[2] <= pare[0]: # 0 -cpm, 1 - start, 2 - end
                continue
            if publ_ad[1] <= pare[0] and publ_ad[2] >= pare[1]:
                scopes[it] = (pare[1], pare[1])
                continue
            if publ_ad[1] > pare[0] and publ_ad[2] < pare[1]:
                new_scope = True
                loc = it
                scopes.append((pare[0], publ_ad[1]))
                scopes.append((publ_ad[2] + 1, pare[1]))
                break
            if publ_ad[1] > pare[0]:
                scopes[it] = (pare[0], publ_ad[1])
            elif publ_ad[2] < pare[1]: # same as used_ad[1] < pare[0]
                scopes[it] = (publ_ad[2], pare[1])
            else:
                print(scopes, publ_ad, (publ_ad[1] <= pare[0] and publ_ad[2] >= pare[1]))
                assert(False)
        if new_scope:
            scopes[-1], scopes[loc] = scopes[loc], scopes[-1]
            scopes.pop()
        if publ == 2:
            print("new scopes ", scopes)
    for pare in scopes:
        sum_hours += pare[1] - pare[0]
    times.append(sum_hours)
time.time() - start_time

cpm -  130.0 start - 957 end -  1379
old scopes  [(957, 1379)]
publ_ad  70.0 1229 1249
new scopes  [(957, 1379)]
old scopes  [(957, 1379)]
publ_ad  280.0 1029 1070
new scopes  [(1071, 1379), (957, 1029)]
old scopes  [(1071, 1379), (957, 1029)]
publ_ad  270.0 978 994
new scopes  [(1071, 1379), (995, 1029), (957, 978)]
old scopes  [(1071, 1379), (995, 1029), (957, 978)]
publ_ad  250.0 1440 1442
new scopes  [(1071, 1379), (995, 1029), (957, 978)]
old scopes  [(1071, 1379), (995, 1029), (957, 978)]
publ_ad  80.0 1031 1040
new scopes  [(1071, 1379), (995, 1029), (957, 978)]
old scopes  [(1071, 1379), (995, 1029), (957, 978)]
publ_ad  55.0 958 1096
new scopes  [(1071, 1379), (995, 1029), (957, 978)]
old scopes  [(1071, 1379), (995, 1029), (957, 978)]
publ_ad  89.0 1328 1339
new scopes  [(1071, 1379), (995, 1029), (957, 978)]
old scopes  [(1071, 1379), (995, 1029), (957, 978)]
publ_ad  212.0 929 961
new scopes  [(1071, 1379), (995, 1029), (961, 978)]
old scopes  [(1071, 1379), (995, 1029), (9

old scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (1114, 1114)]
publ_ad  146.0 1015 1023
new scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (1114, 1114)]
old scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (1114, 1114)]
publ_ad  380.0 858 917
new scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (1114, 1114)]
old scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (1114, 1114)]
publ_ad  30.0 1060 1254
new scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (1114, 1114)]
old scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (1114, 1114)]
publ_ad  30.0 1444 1478
new scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (1114, 1114)]
old scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (1114, 1114)]
publ_ad  40.0 992 1234
new scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (1114, 1114)]
old scopes  [(1379, 1379), (1029, 1029), (978, 978), (1001, 1001), (111

0.3206918239593506

In [11]:
start_time = time.time()
time_for_publishers = []
for i, row in validate.iterrows():
    users_in_row = list(map(int, (row.user_ids.split(','))))
    publishers_in_row = list(map(int, (row.publishers.split(','))))
    n_publish = len(publishers_in_row)
    start = row.hour_start
    end = row.hour_end
    cpm = row.cpm
    times = []
    for publ in publishers_in_row:
        sum_hours = 0
        scopes = [(start, end)]
        for publ_ad in publisher_in_ad[publ]:
            if 
            new_scope = False
            loc = None
            for it, pare in enumerate(scopes):
                if publ_ad[0] <= cpm: # case of == 
                    continue
                if publ_ad[1] >= pare[1] or publ_ad[2] <= pare[0]: # 0 -cpm, 1 - start, 2 - end
                    continue
                if publ_ad[1] <= pare[0] and publ_ad[2] >= pare[1]:
                    scopes[it] = (pare[1], pare[1])
                    continue
                if publ_ad[1] > pare[0] and publ_ad[2] < pare[1]:
                    new_scope = True
                    loc = it
                    scopes.append((pare[0], publ_ad[1]))
                    scopes.append((publ_ad[2] + 1, pare[1]))
                    break
                if publ_ad[1] > pare[0]:
                    scopes[it] = (pare[0], publ_ad[1])
                elif publ_ad[2] < pare[1]: # same as used_ad[1] < pare[0]
                    scopes[it] = (publ_ad[2], pare[1])
                else:
                    print(scopes, publ_ad, (publ_ad[1] <= pare[0] and publ_ad[2] >= pare[1]))
                    assert(False)
            if new_scope:
                scopes[-1], scopes[loc] = scopes[loc], scopes[-1]
                scopes.pop()
        for pare in scopes:
            sum_hours += pare[1] - pare[0]
        times.append(sum_hours)
    time_for_publishers.append(times)
time.time() - start_time

1.2028567790985107

In [40]:
time_for_publishers[52]

[0, 23, 0]

In [19]:
pd.Series(time_for_publishers)

0                   [0, 0]
1                   [0, 0]
2       [0, 0, 0, 0, 0, 0]
3                  [0, 70]
4             [0, 3, 3, 6]
               ...        
1003       [0, 0, 0, 0, 0]
1004                [0, 0]
1005                [0, 0]
1006             [0, 0, 0]
1007          [0, 1, 0, 2]
Length: 1008, dtype: object

In [165]:
df.cpm = (df.cpm - df.cpm.mean()) / df.cpm.std()
df.hours = (df.hours - df.hours.mean()) / df.hours.std()
#df['index'] = df.index

In [164]:
df.head()

,cpm,hour_start,hour_end,publishers,audience_size,user_ids,hours,at_least_one,at_least_two,at_least_three,publ_fit_users,users_overall_activity,estim_activity,average_fit,average_active
0,220.0,1058,1153,"7,17",1906,"12,44,46,50,58,71,93,122,134,143,176,184,187,1...",95,0.0430,0.0152,0.0073,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0006720430107526882, 0.013440860215053764, ...","[0.06384408602150538, 1.2768817204301075, 0.0,...",0.077551,0.027263
1,312.0,1295,1301,"3,18",1380,"29,81,98,102,165,167,195,205,218,231,242,263,3...",6,0.0130,0.0000,0.0000,"[0.1702127659574468, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.031586021505376344, 0.1364247311827957, 0.0...","[0.18951612903225806, 0.8185483870967741, 0.36...",0.053528,0.086687
2,70.0,1229,1249,"1,2,3,9,15,21",888,"12,23,25,29,45,85,92,124,156,190,272,334,456,5...",20,0.0878,0.0135,0.0000,"[0.0, 1.0000000000000002, 0.9999999999999996, ...","[0.0006720430107526882, 0.03696236559139785, 0...","[0.013440860215053765, 0.739247311827957, 0.32...",0.761266,0.029555
3,240.0,1295,1377,"1,14",440,"44,122,187,209,242,255,312,345,382,465,513,524...",82,0.2295,0.1295,0.0727,"[1.0000000000000002, 0.9999999999999987, 1.000...","[0.013440860215053764, 0.07056451612903226, 0....","[1.1021505376344085, 5.786290322580646, 0.9919...",0.479130,0.025985
4,262.0,752,990,"1,3,7,8",1476,"15,24,30,43,50,53,96,105,159,168,181,190,196,2...",238,0.3963,0.2785,0.2270,"[1.0, 0.9074074074074069, 1.0000000000000002, ...","[0.0020161290322580645, 0.036290322580645164, ...","[0.4798387096774193, 8.63709677419355, 5.75806...",0.660492,0.023874


In [77]:
pd.DataFrame({'at_least_one':df.at_least_one, 'at_least_two':df.at_least_two, 'at_least_three':df.at_least_three,\
              'diff':df.at_least_one - df.at_least_three, 'cpm':df.cpm, 'hours':df.hours, \
             'average_active':average_active, 'average_fit':average_fit}).corr()

,at_least_one,at_least_two,at_least_three,diff,cpm,hours,average_active,average_fit
at_least_one,1.000000,0.962787,0.918242,0.815272,0.199448,0.480316,0.358690,0.548116
at_least_two,0.962787,1.000000,0.986001,0.635886,0.150554,0.534585,0.343946,0.425385
at_least_three,0.918242,0.986001,1.000000,0.519290,0.132197,0.547955,0.352876,0.371852
diff,0.815272,0.635886,0.519290,1.000000,0.237096,0.235262,0.258051,0.639077
cpm,0.199448,0.150554,0.132197,0.237096,1.000000,-0.011378,0.003511,0.001494
hours,0.480316,0.534585,0.547955,0.235262,-0.011378,1.000000,0.060703,-0.021334
average_active,0.358690,0.343946,0.352876,0.258051,0.003511,0.060703,1.000000,0.089754
average_fit,0.548116,0.425385,0.371852,0.639077,0.001494,-0.021334,0.089754,1.000000


In [264]:
final_df = df[['cpm', 'hours', 'users_overall_activity', 'publ_fit_users',\
               #'user_ids', 'index',\
               'at_least_one','at_least_two', 'at_least_three']].to_numpy()
num_featers = 4
extra_featers = 0 # that adds in extand
info = 2
X, y = final_df[:, 0:num_featers], final_df[:, num_featers:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)
y_train = torch.Tensor(y_train.astype('float32')).float()
y_test = torch.Tensor(y_test.astype('float32')).float()

In [265]:
def expand_df(cur_df):
    start = time.time()
    ids = []
    series = []
    current_id = 0
    for i in range(cur_df.shape[0]):
        """
        s = cur_df[i][6]
        t = cur_df[i][7]
        user_ids = cur_df[i][5].split(',')
        for j in range(len_):
            cur_user = int(user_ids[j])
            
            cpms = []
            for ad in user_in_ad[cur_user]: # 0 - cpm, 1 - start, 2 - end
                crossing = min(ad[2] - s, t - ad[1])
                #crossing = min(ad.hour_end - s, t - ad.hour_start)
                if (4 * crossing >= t - s):
                    cpms.append(ad[0])
            cpms = np.array(cpms)
            final_cpm = None
            if cpms.size == 1:
                final_cpm = 2
            elif cur_df[i][0] == cpms[-1] and cpms[-1] == cpms[-2]:
                final_cpm = 1.5
            else:
                final_cpm = (cur_df[i][0] - cpms.mean()) / cpms.std()
            """    
        len_ = cur_df[i][2].size
        #user_ids = cur_df[i][4].split(',')
        #index = cur_df[i][5]
        for j in range(len_):
            #user_will_see_hours = ad_user_hours[(index, int(user_ids[j]))]
            series.append([cur_df[i][0], cur_df[i][1], cur_df[i][2][j], cur_df[i][3][j]])
        ids.append((current_id, current_id + len_))
        current_id += len_
    series = np.array(series)
    ids = pd.Series(ids)
    print(time.time() - start)
    return series, ids

In [266]:
start = time.time()
X_train, train_ids = expand_df(X_train)
X_test, test_ids = expand_df(X_test)
time.time() - start

2.1404130458831787
0.6190862655639648


2.760314702987671

In [244]:
pd.DataFrame({'uwsh':X_train[:, 0], 'res':X_train[:, -1]}).corr()

,uwsh,res
uwsh,1.000000,0.394932
res,0.394932,1.000000


In [267]:
X_train.shape

(879020, 4)

In [268]:
check_none = pd.DataFrame(X_train)
check_none.isna().sum().sum()

0

In [269]:
X_train[:, 2:] = scale(X_train[:, 2:])
X_test[:, 2:] = scale(X_test[:, 2:])

In [270]:
X_train.shape

(879020, 4)

In [271]:
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)

In [272]:
class Net(nn.Module):
    def __init__(self, hidden_sizes):
        super(Net, self).__init__()
        self.input_size = num_featers + extra_featers
        self.output_size = 3
        self.layers = nn.Sequential(
            
            nn.Linear(self.input_size, hidden_sizes[0]),
            #nn.BatchNorm1d(hidden_sizes[0]),
            nn.ReLU(),

            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            #nn.BatchNorm1d(hidden_sizes[1]),
            nn.ReLU(),

            nn.Linear(hidden_sizes[1], hidden_sizes[2]),
            #nn.BatchNorm1d(hidden_sizes[2]),
            nn.ReLU(),
            
            nn.Linear(hidden_sizes[2], hidden_sizes[3]),
            #nn.BatchNorm1d(hidden_sizes[3]),
            nn.ReLU(),
            
            #nn.Linear(hidden_sizes[3], hidden_sizes[4]),
            #nn.BatchNorm1d(hidden_sizes[3]),
            #nn.ReLU(),
            
            nn.Linear(hidden_sizes[3], self.output_size),
            nn.Sigmoid()
          )
  
    def forward(self, x):
        return self.layers(x)
    
def getitem(X, y, ids, index):
    left, right = ids[index]
    return X[left:right, :], y[index]
    
def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
    return torch.abs(torch.log((responses_column + epsilon) / (answers_column + epsilon))).mean()


def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):
    log_accuracy_ratio_mean = torch.stack(
        [
            get_smoothed_log_mape_column_value(responses[:, 0], answers[:, 0], epsilon),
            get_smoothed_log_mape_column_value(responses[:, 1], answers[:, 1], epsilon),
            get_smoothed_log_mape_column_value(responses[:, 2], answers[:, 2], epsilon),
        ]
    ).mean()
    percentage_error = 100 * (torch.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error


def train_(responses_column, answers_column, epsilon):
    return torch.abs(torch.log((responses_column + epsilon) / (answers_column + epsilon))).mean()

def loss_for_train(answers, responses, epsilon=0.005):
    log_accuracy_ratio_mean = torch.stack(
        [
            train_(responses[:, 0], answers[:, 0], epsilon),
            train_(responses[:, 1], answers[:, 1], epsilon),
            train_(responses[:, 2], answers[:, 2], epsilon),
        ]
    ).mean()
    
    return log_accuracy_ratio_mean

In [273]:
model = Net([10, 10, 10, 10])
opt = optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.0001) #1e-2, 0.01 - 74; 5e-3, 0.01 - 
loss_fun = get_smoothed_mean_log_accuracy_ratio#nn.MSELoss()

In [235]:
h = (check_df.hours - check_df.hours.mean()) / check_df.hours.std()
g = (check_df.cpm - check_df.cpm.mean()) / check_df.cpm.std()
a = check_df.average_active.to_numpy()
a = (a - a.mean()) / a.std()
t = check_df.average_fit.to_numpy()#.reshape(1, -1)
s = check_df.at_least_one.to_numpy()#.reshape(1, -1)
t.shape, s.shape
X = list(map(list, zip(a, t, h, g)))
y = s
reg = LinearRegression().fit(X, y)
print(((t - s) ** 2).mean())
pred = np.array([reg.predict([input_]) for input_ in map(list, zip(a, t, h, g))])
print(((pred - s[:, np.newaxis]) ** 2).mean())
reg.score(X, y)

0.12849833767902089
0.007202414137923546


0.6624542265651655

In [236]:
s

array([0.043 , 0.013 , 0.0878, ..., 0.0071, 0.3669, 0.071 ])

In [239]:
((pred < 0) | (pred > 1)).sum()

189

In [211]:
pred = np.array([reg.predict([input_]) for input_ in map(list, zip(a))])
print(pred.shape)
(t[:, np.newaxis] + pred).shape

(1008, 1)


(1008, 1)

In [196]:
((t[:, np.newaxis] + pred) < 0).sum()

413

In [274]:
batch_size = 24
epochs = 40
test_loss = []
train_loss = []
for i in range(epochs):
    order = np.random.permutation(y_train.shape[0])
    batch_loss = torch.Tensor(batch_size, 3)
    target_scores = torch.Tensor(batch_size, 3)
    # check_loss = []
    for it, j in enumerate(order):
        X_current, y_current = getitem(X_train, y_train, train_ids, j)
        scores = model(X_current)
        batch_loss[it % batch_size] = scores.mean(axis=0)
        target_scores[it % batch_size] = y_current
        if (it + 1) % batch_size == 0:
            opt.zero_grad()
            loss = get_smoothed_mean_log_accuracy_ratio(target_scores, batch_loss) # for custom
            #loss = loss_fun(batch_loss, target_scores) # for mse
            batch_loss = torch.Tensor(batch_size, 3)
            target_scores = torch.Tensor(batch_size, 3)
            loss.backward()
            opt.step()
        
    epoch_loss = torch.Tensor(y_train.shape[0], 3)
    for j in range(y_train.shape[0]):
        X_current, y_current = getitem(X_train, y_train, train_ids, j)
        scores = model(X_current)
        epoch_loss[j] = scores.mean(axis=0)
        
    train_loss.append(int(get_smoothed_mean_log_accuracy_ratio(y_train, epoch_loss).item()))
    
    epoch_loss = torch.Tensor(y_test.shape[0], 3)
    for j in range(y_test.shape[0]):
        X_current, y_current = getitem(X_test, y_test, test_ids, j)
        scores = model(X_current)
        epoch_loss[j] = scores.mean(axis=0)
    test_loss.append(int(get_smoothed_mean_log_accuracy_ratio(y_test, epoch_loss).item()))
print(train_loss)
print(test_loss)

[255, 114, 84, 64, 52, 47, 41, 37, 35, 34, 33, 33, 33, 31, 31, 30, 28, 27, 26, 25, 24, 25, 28, 25, 23, 22, 23, 23, 26, 22, 22, 22, 22, 22, 24, 23, 23, 25, 23, 23]
[267, 120, 87, 68, 57, 51, 44, 40, 37, 36, 35, 34, 34, 33, 35, 31, 31, 31, 29, 26, 25, 26, 28, 25, 24, 23, 25, 24, 27, 23, 23, 23, 23, 24, 24, 24, 22, 27, 25, 24]


In [ ]:
# (22, 22) 12 12 12(tanh bag) 12 1e-2 0.0001

In [ ]:
#(22, 24) ReLU 35 10 10 10 1e-2 0.0001

In [ ]:
# (23, 25) - cpm,hours,users_overall_activity,publ_fit_users; 10, 10, 10; 2e-2, 0.0001

In [ ]:
# no time, to cpm, but hours_of_user_will_see_this_ad sum in publics weighted by users_publ. 
# 5 featers; 12 12 12 12 8e-3 0.025 - (30, 32) can be less

In [ ]:
# 5 featers(no cpm, no hours, but user_will_see_this_ad) => 36 :(

In [375]:
# bs-32, 10 10 10, 9e-3, 0.005 - 71
# 10 10 10 10, 9e-3, 0.0001 - 72
# 12 12 12 12, 9e-3, 0.00001 - 70
# 6 featers - 76 :(
# user_publisher - not unique - 45 (10, 10, 10; 1e-2; 0.0001) overfit => reg 0.01 - 38
# cpm+hours+up => 51
# add over_all_active => 36! (might be a bit lucky)
# add age => 35; changed reg to 0.02 and echoches to 25 => 36 jumpes over min => lr t0 8e-3 from 1e-2 => 33
# add sex => not really better, 35 and jumps over the min

In [ ]:
# check age_publish for unique users (without sex; 10, 10, 10; 1e-2; 0.01; 25 - 30(37) but very noise, not the best
# multiple hours and active: (age is unique hear and not unique in one hour)
# no hours, no overall, but * => 38-41; all 6(without sex) - 32-34, 30 epiches - 31; not unique users in age - 32
# model size - 6featers(no sex) not unique age, not unique hours, 12, 12, 12, 12; 8e-3; 0.025 30epoches => 28!
# boosting :)
# active - not unique in one hour 36 and 38 with 'extra' norm

In [54]:
model(getitem(X_train, y_train, train_ids, 4)[0]).mean(axis=0)

tensor([0.3018, 0.2617, 0.1664], grad_fn=<MeanBackward1>)

In [55]:
getitem(X_train, y_train, train_ids, 4)[1]

tensor([0.6085, 0.4690, 0.3129])

In [21]:
final_df = df[['cpm', 'publ_fit_users', 'estim_activity', 'at_least_one', 'at_least_two', 'at_least_three']].copy()
num_featers = 3
X, y = final_df.iloc[:, :num_featers].to_numpy(), final_df.iloc[:, num_featers:].to_numpy()
#X[:, 0] = (X[:, 0] - X[:, 0].mean()) / X[:, 0].std()

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = torch.tensor(y_train).float()
y_test = torch.tensor(y_test).float()

In [26]:
torch.Tensor([1, 2, 3])

tensor([1., 2., 3.])

In [61]:
tmp = torch.Tensor([X_train[0][0], X_train[0][1][0], X_train[0][2][0]])
tmp2 = torch.Tensor([[X_train[0][0], X_train[0][1][0], X_train[0][2][0]],\
                    [X_train[0][0], X_train[0][1][1], X_train[0][2][1]]])
print(model(tmp))
model(tmp2)

tensor([0.5797, 0.5373, 0.6009], grad_fn=<SigmoidBackward>)


tensor([[0.5797, 0.5373, 0.6009],
        [0.5793, 0.5393, 0.5996]], grad_fn=<SigmoidBackward>)

In [ ]:
# модель принимает audience_size sample'ов каждый раз